In [1]:
import sys  
sys.path.insert(0, '/Users/pratikaher/DGL/GRecSy/')

from random import sample
import dgl
import torch
import pandas as pd
import numpy as np
import torch
from grecsy_hetero.model import compute_loss

from dgl import AddReverse
transform = AddReverse()

df_rating = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'],  engine='python')
df_user = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/users.dat', sep='::', header=None, names=['user_id', 'gender', 'age', 'occupation', 'zipcode'],  engine='python')
df_movie = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genre'],  engine='python')

/Users/pratikaher/DGL/graph-rec/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_temp = df_rating.merge(df_movie, left_on='movie_id', right_on='movie_id', how='left')
df_final = df_temp.merge(df_user, left_on='user_id', right_on='user_id', how='left')

In [5]:
df_final.shape

(1000209, 10)

In [6]:
df_final['movie_id'].nunique()

3706

In [7]:
df_final['user_id'].nunique()

6040

In [8]:
df_final['release_year'] = df_final.title.str.extract("\((\d{4})\)", expand=True).astype(str)
df_final['release_year'] = pd.to_datetime(df_final.release_year, format='%Y')
df_final['release_year'] = df_final.release_year.dt.year
df_final['title'] = df_final.title.str[:-7]

In [9]:
df_final.head()

,user_id,movie_id,rating,timestamp,title,genre,gender,age,occupation,zipcode,release_year
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest,Drama,F,1,10,48067,1975
1,1,661,3,978302109,James and the Giant Peach,Animation|Children's|Musical,F,1,10,48067,1996
2,1,914,3,978301968,My Fair Lady,Musical|Romance,F,1,10,48067,1964
3,1,3408,4,978300275,Erin Brockovich,Drama,F,1,10,48067,2000
4,1,2355,5,978824291,"Bug's Life, A",Animation|Children's|Comedy,F,1,10,48067,1998


In [108]:
hetero_graph = dgl.heterograph({('user', 'rates', 'movie'): (df_final['user_id'].to_numpy(), df_final['movie_id'].to_numpy())})

In [109]:
hetero_graph.nodes(ntype='movie')

tensor([   0,    1,    2,  ..., 3950, 3951, 3952])

In [110]:
hetero_graph = transform(hetero_graph)

In [51]:
hetero_graph.ndata

defaultdict(<class 'dict'>, {})

In [111]:
hetero_graph

Graph(num_nodes={'movie': 3953, 'user': 6041},
      num_edges={('movie', 'rev_rates', 'user'): 1000209, ('user', 'rates', 'movie'): 1000209},
      metagraph=[('movie', 'user', 'rev_rates'), ('user', 'movie', 'rates')])

In [112]:
df_movie['release_year'] = df_movie.title.str.extract("\((\d{4})\)", expand=True).astype(str)
df_movie['release_year'] = pd.to_datetime(df_movie.release_year, format='%Y')
df_movie['release_year'] = df_movie.release_year.dt.year
df_movie['title'] = df_movie.title.str[:-7]

In [66]:
t = list(range(0, 3953))

In [76]:
required = [x for x in t if x not in isolated_nodes.tolist()]

In [98]:
hetero_graph.nodes('movie')

tensor([   0,    1,    2,  ..., 3950, 3951, 3952])

In [93]:
hetero_graph.nodes('movie')[required].data['features'] = torch.unsqueeze(torch.tensor(df_movie['release_year'].to_numpy()), dim = 1).float()

IndexError: too many indices for tensor of dimension 1

In [113]:
isolated_nodes = ((hetero_graph.in_degrees(etype='rates') == 0) & (hetero_graph.out_degrees(etype='rev_rates') == 0)).nonzero().squeeze(1)

In [58]:
isolated_nodes = ((hetero_graph.in_degrees(etype='rev_rates') == 0) & (hetero_graph.out_degrees(etype='rates') == 0)).nonzero().squeeze(1)

In [41]:
pd.set_option('display.max_rows', None)

In [114]:
isolated_nodes = ((hetero_graph.in_degrees(etype='rates') == 0) & (hetero_graph.out_degrees(etype='rev_rates') == 0)).nonzero().squeeze(1)
hetero_graph = dgl.remove_nodes(hetero_graph, isolated_nodes, ntype='movie')

isolated_nodes = ((hetero_graph.in_degrees(etype='rev_rates') == 0) & (hetero_graph.out_degrees(etype='rates') == 0)).nonzero().squeeze(1)
hetero_graph = dgl.remove_nodes(hetero_graph, isolated_nodes, ntype='user')

In [115]:
hetero_graph

Graph(num_nodes={'movie': 3706, 'user': 6040},
      num_edges={('movie', 'rev_rates', 'user'): 1000209, ('user', 'rates', 'movie'): 1000209},
      metagraph=[('movie', 'user', 'rev_rates'), ('user', 'movie', 'rates')])

In [116]:
hetero_graph.edges['rates'].data['rating'] =  torch.unsqueeze(torch.tensor(df_final['rating'].to_numpy()), dim = 1)

In [118]:
hetero_graph.nodes['user'].data['features'] = torch.unsqueeze(torch.tensor(df_user['age'].to_numpy()), dim = 1).float()

In [122]:
df_final['movie_id'].nunique()

3706

In [128]:
hetero_graph.successors(100, etype='rev_rates').tolist()

[523,
 686,
 709,
 751,
 969,
 1050,
 1679,
 1967,
 2503,
 2535,
 2742,
 2776,
 2908,
 3031,
 3083,
 3120,
 3376,
 3390,
 3490,
 3506,
 3525,
 3538,
 3602,
 3835,
 3840,
 4168,
 4226,
 4276,
 4403,
 5038,
 5248,
 5268,
 5941]

In [133]:
df_final[df_final['user_id'] == 10]

,user_id,movie_id,rating,timestamp,title,genre,gender,age,occupation,zipcode,release_year
799,10,2622,5,978228212,"Midsummer Night's Dream, A",Comedy|Fantasy,F,35,1,95370,1999
800,10,648,4,978224925,Mission: Impossible,Action|Adventure|Mystery,F,35,1,95370,1996
801,10,2628,3,978228408,Star Wars: Episode I - The Phantom Menace,Action|Adventure|Fantasy|Sci-Fi,F,35,1,95370,1999
802,10,3358,5,978226378,Defending Your Life,Comedy|Romance,F,35,1,95370,1991
803,10,3359,3,978227125,Breaking Away,Drama,F,35,1,95370,1979
804,10,1682,5,978226319,"Truman Show, The",Drama,F,35,1,95370,1998
805,10,1756,4,978228655,"Prophecy II, The",Horror,F,35,1,95370,1998
806,10,1320,3,978230837,Alien�,Action|Horror|Sci-Fi|Thriller,F,35,1,95370,1992
807,10,2124,3,978229208,"Addams Family, The",Comedy,F,35,1,95370,1991
808,10,2125,5,979167795,Ever After: A Cinderella Story,Drama|Romance,F,35,1,95370,1998


In [81]:
df_temp.shape

(1000386, 6)

In [54]:
df_temp['release_year'] = df_temp.title.str.extract("\((\d{4})\)", expand=True).astype(str)
df_temp['release_year'] = pd.to_datetime(df_temp.release_year, format='%Y')
df_temp['release_year'] = df_temp.release_year.dt.year
df_temp['title'] = df_temp.title.str[:-7]

In [4]:
hetero_graph = dgl.heterograph({('user', 'rates', 'movie'): (df_final['user_id'].to_numpy(), df_final['movie_id'].to_numpy())})

In [5]:
# hetero_graph.nodes['movie'].data['duration'] = torch.tensor(df_user['age'].values).float()

In [6]:
hetero_graph.remove_nodes(torch.tensor([0]), ntype='user')

In [7]:
import torch
hetero_graph.nodes['user'].data['age'] = torch.unsqueeze(torch.tensor(df_user['age'].to_numpy()), dim = 1)

In [8]:
# Fix this to include actual release years
# hetero_graph.nodes['movie'].data['release_year'] = torch.tensor(df_movie['release_year'].values).float()
hetero_graph.nodes['movie'].data['release_year'] = torch.randn(3953, 1).float()
hetero_graph.edges['rates'].data['rating'] =  torch.unsqueeze(torch.tensor(df_final['rating'].to_numpy()), dim = 1)

In [51]:
hetero_graph.nodes['movie'].data['release_year'] = torch.unsqueeze(torch.tensor(df_final['release_year'].to_numpy()), dim = 1)

DGLError: Expect number of features to match number of nodes (len(u)). Got 1000209 and 3953 instead.

In [67]:
len(df_movie['title'].to_numpy())

3883

In [21]:
torch.tensor(df_movie['release_year'].values).float().shape

torch.Size([3883])

In [9]:
# isolated_nodes = ((g.in_degrees() == 0) & (g.out_degrees() == 0)).nonzero().squeeze(1)
# g = dgl.remove_nodes(g, isolated_nodes)

In [10]:
# hetero_graph.nodes['user'].data['age'] = torch.randn(6041, 1).float()
# hetero_graph.nodes['movie'].data['duration'] = torch.randn(3953, 1).float()
# hetero_graph.edges['rates'].data['rating'] = torch.randn(1000209, 1)

In [ ]:
# Add reverse edges
# from dgl import AddReverse
# transform = AddReverse()

In [11]:
hetero_graph

Graph(num_nodes={'movie': 3953, 'user': 6040},
      num_edges={('user', 'rates', 'movie'): 1000209},
      metagraph=[('user', 'movie', 'rates')])

In [62]:
hetero_graph.nodes['user'].data['age']

tensor([ 1., 56., 25.,  ..., 56., 45., 25.])

In [5]:
# def construct_negative_graph(graph, k, etype):
#     utype, _, vtype = etype
#     src, dst = graph.edges(etype=etype)
#     neg_src = src.repeat_interleave(k)
#     neg_dst = torch.randint(0, graph.num_nodes(vtype), (len(src) * k,))
#     return dgl.heterograph(
#         {etype: (neg_src, neg_dst)},
#         num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

In [6]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, etype):
        h = self.sage(g, x)
        return self.pred(g, h, etype), self.pred(neg_g, h, etype)

In [7]:
from dgllife.model.gnn import MPNNGNN
import dgl.function as fn

class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        # h contains the node representations for each node type computed from
        # the GNN defined in the previous section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']
        
class MPNNPredictor(nn.Module):
    def __init__(self,
                 node_in_feats,
                 edge_in_feats,
                 node_out_feats=64,
                 edge_hidden_feats=128,
                 n_tasks=1,
                 num_step_message_passing=6):
        super(MPNNPredictor, self).__init__()

        self.gnn = MPNNGNN(node_in_feats=node_in_feats,
                           node_out_feats=node_out_feats,
                           edge_in_feats=edge_in_feats,
                           edge_hidden_feats=edge_hidden_feats,
                           num_step_message_passing=num_step_message_passing)
        self.predictor = HeteroDotProductPredictor()
    
    def forward(self, g, node_feats, edge_feats, node_subgraph_negative):
        node_feats = self.gnn(g, node_feats, edge_feats)
        return self.predictor(g, node_feats, ('user', 'rates', 'movie')), self.predictor(node_subgraph_negative, node_feats)
    
    def get_repr(self, node_features):
        """Returns the embedded representation given block made from sampling neighboring nodes."""
        
        node_feats = self.gnn(g, node_feats, edge_feats)
        return node_feats
        

In [12]:
mpnn_model = MPNNPredictor(hetero_graph.ndata['age']['user'].shape[1], hetero_graph.edata['rating'].shape[1])

NameError: name 'MPNNPredictor' is not defined

In [8]:
# user_feats = hetero_graph.nodes['user'].data['age']
# item_feats = hetero_graph.edges['rates'].data['rating']
# negative_graph = construct_negative_graph(hetero_graph, 5, ('user', 'rates', 'movie'))
# pos_score, neg_score = mpnn_model(hetero_graph, user_feats, item_feats, negative_graph)

In [56]:
k = 5
user_feats = hetero_graph.nodes['user'].data['age']
item_feats = hetero_graph.nodes['movie'].data['duration']
node_features = {'user': user_feats, 'item': item_feats}
opt = torch.optim.Adam(model.parameters())
for epoch in range(10):
    negative_graph = construct_negative_graph(hetero_graph, k, ('user', 'rates', 'movie'))
    pos_score, neg_score = mpnn_model(hetero_graph, user_feats, item_feats, negative_graph)
    loss = compute_loss(pos_score, neg_score)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

RuntimeError: random_ expects 'from' to be less than 'to', but got from=0 >= to=0

In [55]:
hetero_graph.ndata

defaultdict(<class 'dict'>, {'duration': {'movie': tensor([], size=(0, 1))}, '_ID': {'movie': tensor([], dtype=torch.int64), 'user': tensor([], dtype=torch.int64)}, 'age': {'user': tensor([], size=(0, 1))}})

In [9]:
# hetero_graph = dgl.in_subgraph(hetero_graph, {'dst_type': 0}, relabel_nodes=True)

In [86]:
u = [0, 1, 0, 0, 1]
v = [0, 1, 2, 3, 2]
# hg = dgl.heterograph((np.array(u), np.array(v)))
hg = dgl.heterograph({('user', 'rates', 'movie'): (np.array(u), np.array(v)),
                     })
u_feat = torch.tensor(np.random.rand(2, 10).astype(np.float32))

In [67]:
hg

Graph(num_nodes={'movie': 4, 'user': 2},
      num_edges={('user', 'rates', 'movie'): 5},
      metagraph=[('user', 'movie', 'rates')])

In [35]:
from dgl import AddReverse

In [36]:
transform = AddReverse()

In [38]:
hetero_graph = transform(hetero_graph)

In [39]:
hetero_graph

Graph(num_nodes={'movie': 3953, 'user': 6040},
      num_edges={('movie', 'rev_rates', 'user'): 1000209, ('user', 'rates', 'movie'): 1000209},
      metagraph=[('movie', 'user', 'rev_rates'), ('user', 'movie', 'rates')])

In [72]:
v_feat = torch.tensor(np.random.rand(4, 10).astype(np.float32))

In [73]:
v_feat.shape

torch.Size([4, 10])

In [76]:
efeat = torch.ones(5, 5)

In [77]:
efeat

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [91]:
hg2

Graph(num_nodes={'movie': 4, 'user': 2},
      num_edges={('movie', 'rev_rates', 'user'): 5, ('user', 'rates', 'movie'): 5},
      metagraph=[('movie', 'user', 'rev_rates'), ('user', 'movie', 'rates')])

In [81]:
from dgl.nn import NNConv
conv = NNConv(10, 2, edge_func, 'mean')

In [82]:
res = conv(hg2, (u_feat, v_feat), efeat)

AssertionError: Current HeteroNodeDataView has multiple node types, please passing the node type and the corresponding data through a dict.

In [79]:
lin = torch.nn.Linear(10, 20)
def edge_func(efeat):
    return lin(efeat)

In [49]:
res

tensor([[-0.5243, -0.6964],
        [ 0.0350,  0.0194],
        [-0.2446, -0.3385],
        [-0.5243, -0.6964]], grad_fn=<AddBackward0>)

In [107]:
%load_ext autoreload
%autoreload 2

In [94]:
hetero_graph.nodes['user'].data

{'age': tensor([ 1., 56., 25.,  ..., 56., 45., 25.])}

In [10]:
from grecsy_hetero.model import MPNNConvModel
params = {
'hidden_dim' : 128,
'out_dim' : 64
}
dim_dict = {'user': hetero_graph.nodes['user'].data['age'].shape[1],
            'movie': hetero_graph.nodes['movie'].data['release_year'].shape[1],
            'out': params['out_dim'],
            'hidden': params['hidden_dim']}

In [11]:
train_eids_dict = {}

for e in hetero_graph.etypes:
    train_eids = np.arange(hetero_graph.number_of_edges(e))
    train_eids_dict[e] = train_eids

In [12]:
train_eids_dict

{'rates': array([      0,       1,       2, ..., 1000206, 1000207, 1000208])}

In [13]:
# eids = train_eids_dict['rates']
# train_eids_dict['rates'] = eids[int(len(eids) * (1 - purchases_sample)):]

In [14]:
model = MPNNConvModel(hetero_graph, dim_dict)

In [15]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([15, 10, 5])
dataloader = dgl.dataloading.EdgeDataLoader(
        hetero_graph, train_eids_dict, 
        sampler, negative_sampler=dgl.dataloading.negative_sampler.Uniform(5), 
        shuffle=True, drop_last=False,
        batch_size=16
        )

/Users/pratikaher/DGL/graph-rec/venv/lib/python3.8/site-packages/dgl/dataloading/dataloader.py:968: DGLWarning: EdgeDataLoader directly taking a BlockSampler will be deprecated and it will not support feature prefetching. Please use dgl.dataloading.as_edge_prediction_sampler to wrap it.
  dgl_warning(


In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0)

In [18]:
for _, pos_g, neg_g, blocks in dataloader:
    
    optimizer.zero_grad()

    input_features = blocks[0].srcdata['features']
    
    print(len(blocks))

    _, pos_score, neg_score = model(blocks,
                                    input_features,
                                    pos_g,
                                    neg_g,
                                    )

    print(pos_score, neg_score)

3
{}
('user', 'rates', 'movie')


KeyError: 'user'